<a href="https://colab.research.google.com/github/kvamsi7/facial-emotions-recognition/blob/main/facial-emotions-recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 5.01 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [4]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader) # skiping headers

  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]


In [9]:
emotions_list = list(data.keys())
emotions_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [7]:
# Data Directory
import os 

In [8]:
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [10]:
for emotion in emotions_list:
  os.mkdir(os.path.join('master_data/training',emotion))
  os.mkdir(os.path.join('master_data/testing',emotion))

In [13]:
from shutil import copyfile
split_size = 0.8

for emotion,images in data.items():
  train_size = int(split_size * len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]

  for image in train_images:
    source = os.path.join('/content/facial_expressions/images',image)
    dest = os.path.join('/content/master_data/training/',emotion,image)
    copyfile(source,dest)

  for image in test_images:
    source = os.path.join('/content/facial_expressions/images',image)
    dest = os.path.join('/content/master_data/testing/',emotion,image)
    copyfile(source,dest)